# Pre-process

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df_target = pd.read_csv("df_target.csv")

In [ ]:
shap_variables = np.array([
    'Emissions|CO2', 
    'Final Energy|Industry|Solids|Coal', 
    'Final Energy|Industry|Solids|Biomass',
    'Final Energy|Residential and Commercial|Solids|Coal', 
    'Emissions|CO2|Energy|Demand|Industry',
    'Secondary Energy|Electricity|Coal', 
    'Emissions|CO2|Energy|Supply|Electricity',
    'Primary Energy|Coal', 
    'Emissions|CO2|Energy and Industrial Processes', 
    'Emissions|CO2|AFOLU'
])

In [ ]:
shap_vars_set = set(shap_variables)
num_shap_vars = len(shap_vars_set)

In [ ]:
df_grouped = df_target.groupby(['Model', 'Scenario'])
pair_counts = df_grouped['Variable'].nunique()

In [ ]:
common_pairs_index = []
for index, group in df_grouped:
    variables_in_pair = set(group['Variable'])
    if variables_in_pair >= shap_vars_set:  
        common_pairs_index.append(index)

In [ ]:
if common_pairs_index:
    df_shap = df_target[df_target.set_index(['Model', 'Scenario']).index.isin(common_pairs_index)]
else:
    df_shap = pd.DataFrame(columns=df_target.columns)

In [ ]:
if common_pairs_index:
    df_temp = df_target[df_target.set_index(['Model', 'Scenario']).index.isin(common_pairs_index)]
    df_shap = df_temp[df_temp['Variable'].isin(shap_variables)]
else:
    df_shap = pd.DataFrame(columns=df_target.columns)

In [ ]:
common_pairs_index = pd.MultiIndex.from_tuples(common_pairs_index, names=['Model', 'Scenario'])

In [ ]:
df_shap.to_csv("df_shap.csv", index = False)

In [ ]:
df_paired = df_shap.set_index(['Model', 'Scenario', 'Variable'])

## Constrcy feature matrix

In [ ]:
year_cols = [col for col in df_shap if col.isdigit()]  

In [ ]:
shap_vars = [
        'Emissions|CO2', 'Final Energy|Industry|Solids|Coal',  'Final Energy|Industry|Solids|Biomass', 
        'Final Energy|Residential and Commercial|Solids|Coal', 'Emissions|CO2|Energy|Demand|Industry', 
        'Secondary Energy|Electricity|Coal', 'Emissions|CO2|Energy|Supply|Electricity', 
        'Primary Energy|Coal','Emissions|CO2|Energy and Industrial Processes', 'Emissions|CO2|AFOLU'
        ]  

In [ ]:
all_new_features_list = []

for var in shap_vars:  
    print(f"Process Variable '{var}'...")  
    var_idx = pd.MultiIndex.from_product([common_pairs_index.get_level_values('Model'),  
                                                     common_pairs_index.get_level_values('Scenario'),  
                                                     [var]], names=['Model', 'Scenario', 'Variable'])  

    valid_var_idx = var_idx.intersection(df_paired.index)  
    if valid_var_idx.empty:  
        print(f"Warning: Variable '{var}' is not in the DataFrame index. Skipping...")  
        continue  

    var_data = df_paired.loc[valid_var_idx, year_cols]  
    var_data = var_data.reset_index(level='Variable', drop=True)  
    var_data = var_data.apply(pd.to_numeric, errors='coerce')  
    
    if not year_cols:
        feature1 = pd.Series(0.0, index=var_data.index) 
        print(f"Warning: Variable '{var}' has no year columns in range 2020-2100. Feature 1 (Sum) set to 0.")  
    else:  
        feature1 = var_data[year_cols].sum(axis=1, skipna=True)  
        feature1.name = f"{var} Cumulative"  

    if '2020' in var_data.columns and '2030' in var_data.columns:  
        feature2 = (var_data['2030'].fillna(0) - var_data['2020'].fillna(0)) / 10  
    else:  
        print(f"    Warning: Variable '{var}' missing '2020' or '2030' data, Feature 2 set to NaN.")  
        feature2 = pd.Series(np.nan, index=var_data.index) 
    feature2.name = f"{var} 2020-2030"  

    if '2040' in var_data.columns and '2050' in var_data.columns:  
        feature3 = (var_data['2040'].fillna(0) - var_data['2030'].fillna(0)) / 10  
    else:  
        print(f"    Warning: Variable '{var}' missing '2030' or '2040' data, Feature 3 set to NaN.")  
        feature3 = pd.Series(np.nan, index=var_data.index) 
    feature3.name = f"{var} 2030-2040" 


    var_features_df = pd.concat([feature1, feature2, feature3], axis=1)  
    all_new_features_list.append(var_features_df)   

In [ ]:
X = pd.concat(all_new_features_list, axis=1)  

In [ ]:
xy_map = df_shap.groupby(['Model', 'Scenario'])['PC_m'].first()  

In [ ]:
y = xy_map.loc[X.index]  

# XGBoost

In [ ]:
import seaborn as sns  
import matplotlib.pyplot as plt  
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import re  

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## Fitting XGB

In [ ]:
classifier = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
parameters = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [6, 8, 10],
    'min_child_weight': [ 3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2, 0.3]
}
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Starting GridSearchCV for hyperparameter tuning...")
gridsearch = GridSearchCV(classifier, parameters, cv=cv_strategy, n_jobs=-1, verbose=2, scoring='accuracy')
gridsearch.fit(X, y)

In [ ]:
xgb_best_params = gridsearch.best_params_
xgb_best_estimator = gridsearch.best_estimator_

print(f"\nBest Parameters Found by GridSearchCV: {gridsearch.best_params_}")
print(f"Best cross-validated accuracy score during GridSearchCV: {gridsearch.best_score_:.4f}")

In [ ]:
cv_scores = cross_val_score(xgb_best_estimator, X, y, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
print(f"\nCross-Validation Accuracy Scores for each fold: {cv_scores}")
print(f"Mean Cross-Validation Accuracy: {np.mean(cv_scores):.4f}")
print(f"Standard Deviation of Cross-Validation Accuracy: {np.std(cv_scores):.4f}")

In [ ]:
xgb_classifier = XGBClassifier(**gridsearch.best_params_, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_classifier.fit(X, y)
print("Final model trained successfully on all data.")

In [ ]:
importance = xgb_classifier.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance})
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
print(feature_importance_df)

# SHAP

In [ ]:
import shap  
import matplotlib.pyplot as plt  
import numpy as np  
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns

In [ ]:
explainer = shap.TreeExplainer(xgb_classifier)  
shap_output = explainer(X) 

In [ ]:
if isinstance(shap_output, shap.Explanation):
    raw_vals = shap_output.values
else:
    raw_vals = shap_output

In [ ]:
if isinstance(raw_vals, np.ndarray) and raw_vals.ndim == 3:
    shap_values = [ raw_vals[:, :, i] for i in range(raw_vals.shape[2]) ]
else:
    shap_values = raw_vals

In [ ]:
sns.set_style("whitegrid")
plt.rcParams.update({
    "font.size": 12,
    "figure.dpi": 120,
    "axes.titlesize": 14,
    "axes.labelsize": 4,
    "ytick.labelsize": 2
})
plt.rcParams['font.family'] = ['Microsoft YaHei']      
plt.rcParams['axes.unicode_minus'] = False 

In [ ]:
variables = [
    'Emissions|CO2',
    'Final Energy|Industry|Solids|Coal',
    'Final Energy|Industry|Solids|Biomass',
    'Final Energy|Residential and Commercial|Solids|Coal',
    'Emissions|CO2|Energy|Demand|Industry',
    'Secondary Energy|Electricity|Coal',
    'Emissions|CO2|Energy|Supply|Electricity',
    'Primary Energy|Coal',
    'Emissions|CO2|Energy and Industrial Processes',
    'Emissions|CO2|AFOLU'
]

palette_var = sns.color_palette("tab10", len(variables))
color_map = {v: palette_var[i] for i, v in enumerate(variables)}
color_map['Sum of the rest'] = (1.0, 0.0, 0.0)

In [ ]:
top_k = 15  
feature_names = X.columns.to_list()
class_names = ['P1','P2','P3']

In [ ]:
plt.rcParams['font.family'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")
plt.rcParams.update({
    "font.size": 10,
    "figure.dpi": 120,
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "ytick.labelsize": 8
})

In [ ]:
def color_yticks(ax):
    for lbl in ax.get_yticklabels():
        txt = lbl.get_text()
        if txt == 'Sum of the rest':
            lbl.set_color(color_map['Sum of the rest'])
        else:
            var = txt.split('_')[0]
            lbl.set_color(color_map.get(var, 'black'))

In [ ]:
for idx, cls in enumerate(class_names):
    sv        = shap_values[idx]              
    abs_mean  = np.abs(sv).mean(axis=0)      
    order     = np.argsort(abs_mean)[::-1]
    top_idx   = order[:top_k]
    rest_idx  = order[top_k:]

    cols      = [feature_names[i] for i in top_idx] + ['Sum of the rest']
    shap_mat  = np.concatenate([
        sv[:, top_idx],
        np.abs(sv[:, rest_idx]).sum(axis=1, keepdims=True)
    ], axis=1)
    X_mat     = np.concatenate([
        X.iloc[:, top_idx].values,
        np.abs(sv[:, rest_idx]).sum(axis=1, keepdims=True)
    ], axis=1)

    plt.figure(figsize=(10/1.46,10))
    shap.summary_plot(
        shap_mat, X_mat,
        feature_names=cols,
        plot_type="violin",
        sort=False,        
        show=False
    )
    ax = plt.gca()
    plt.gca().tick_params(axis='y', labelsize = 8)
    for label in ax.get_yticklabels():
        label.set_fontweight('bold')
    color_yticks(ax)   
    ax.set_xlabel(
    "SHAP value (impact on model output)",
    fontsize=12,
    fontweight='bold',
    labelpad=15
    )
    for text in ax.texts:
        if text.get_text() == 'Feature Value':
            text.set_fontweight('bold')
    plt.savefig(f"SHAP of Synthetic {cls}.png", dpi=600, bbox_inches = 'tight')
    plt.tight_layout()
    plt.show()
